# FloPy$_3$

### Demo of netCDF and shapefile capabilities within the export module. 

In [1]:
from __future__ import print_function
import os
import flopy

Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("data")
ml = flopy.modflow.Modflow.load(nam_file,model_ws=model_ws,verbose=True)
print(ml.dis.sr)


Creating new model with name: freyberg
--------------------------------------------------

Parsing the namefile --> data/freyberg.nam
Setting filehandles:

--------------------------------------------------
External unit dictionary:
{51: filename:data/freyberg.hds, filetype:DATA(BINARY), 2: filename:data/freyberg.list, filetype:LIST, 11: filename:data/freyberg.dis, filetype:DIS, 13: filename:data/freyberg.bas, filetype:BAS6, 14: filename:data/freyberg.oc, filetype:OC, 15: filename:data/freyberg.lpf, filetype:LPF, 18: filename:data/freyberg.riv, filetype:RIV, 19: filename:data/freyberg.rch, filetype:RCH, 20: filename:data/freyberg.wel, filetype:WEL, 53: filename:data/freyberg.cbc, filetype:DATA(BINARY), 52: filename:data/freyberg.ddn, filetype:DATA(BINARY), 27: filename:data/freyberg.pcg, filetype:PCG}
--------------------------------------------------

loading dis package file...
   Loading dis package with:
      1 layers, 40 rows, 20 columns, and 1 stress periods
   loading laycbd..

### Some netCDF export capabilities:

#### Export a 2d array

In [3]:
# export a 2d array
ml.dis.top.export(os.path.join(model_ws, 'top.nc'))
ml.dis.top.export(os.path.join(model_ws, 'top.shp'))

('2016-01-16 15:22:25.379722 starting: initializing attributes\n',)
('2016-01-16 15:22:25.379801 finished: initializing attributes, took: 0:00:00.000079\n',)
('2016-01-16 15:22:25.379829 starting: getting 2D array for top\n',)
('2016-01-16 15:22:25.379904 finished: getting 2D array for top, took: 0:00:00.000075\n',)
('2016-01-16 15:22:25.380090 starting: creating variable: top\n',)
('2016-01-16 15:22:25.380141 starting: initializing geometry\n',)
('2016-01-16 15:22:25.381402 starting: building grid crs using proj4 string: +init=epsg:4326\n',)
('2016-01-16 15:22:25.381562 finished: building grid crs using proj4 string: +init=epsg:4326, took: 0:00:00.000160\n',)
('2016-01-16 15:22:25.381912 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x1088a42c0> to <pyproj.Proj object at 0x1088a4390>\n',)
('2016-01-16 15:22:25.381990 finished: projecting grid cell center arrays from <pyproj.Proj object at 0x1088a42c0> to <pyproj.Proj object at 0x1088a4390>, took: 0:00:00.000

#### Export a 3d array

In [4]:
#export a 3d array
ml.lpf.hk.export(os.path.join(model_ws, 'hk.nc'))
ml.lpf.hk.export(os.path.join(model_ws, 'hk.shp'))

('2016-01-16 15:22:40.985996 WARNING: removing existing nc file: data/hk.nc\n',)
('2016-01-16 15:22:40.986296 starting: initializing attributes\n',)
('2016-01-16 15:22:40.986366 finished: initializing attributes, took: 0:00:00.000070\n',)
('2016-01-16 15:22:40.986395 starting: getting 3D array for hk\n',)
('2016-01-16 15:22:40.986482 finished: getting 3D array for hk, took: 0:00:00.000087\n',)
('2016-01-16 15:22:40.986578 starting: creating variable: hk\n',)
('2016-01-16 15:22:40.986625 starting: initializing geometry\n',)
('2016-01-16 15:22:40.986673 starting: building grid crs using proj4 string: +init=epsg:4326\n',)
('2016-01-16 15:22:40.986734 finished: building grid crs using proj4 string: +init=epsg:4326, took: 0:00:00.000061\n',)
('2016-01-16 15:22:40.986856 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x10896e600> to <pyproj.Proj object at 0x1088a4188>\n',)
('2016-01-16 15:22:40.986955 finished: projecting grid cell center arrays from <pyproj.Proj ob

#### Export a number of things to the same netCDF file

In [5]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(model_ws, 'test.nc'))
ml.lpf.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


('2016-01-16 15:22:55.306674 WARNING: removing existing nc file: data/test.nc\n',)
('2016-01-16 15:22:55.307416 starting: initializing attributes\n',)
('2016-01-16 15:22:55.307494 finished: initializing attributes, took: 0:00:00.000078\n',)
('2016-01-16 15:22:55.307525 starting: getting 3D array for botm\n',)
('2016-01-16 15:22:55.307609 finished: getting 3D array for botm, took: 0:00:00.000084\n',)
('2016-01-16 15:22:55.307727 starting: creating variable: botm\n',)
('2016-01-16 15:22:55.307778 starting: initializing geometry\n',)
('2016-01-16 15:22:55.307828 starting: building grid crs using proj4 string: +init=epsg:4326\n',)
('2016-01-16 15:22:55.307890 finished: building grid crs using proj4 string: +init=epsg:4326, took: 0:00:00.000062\n',)
('2016-01-16 15:22:55.308047 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x1088a4188> to <pyproj.Proj object at 0x10896e598>\n',)
('2016-01-16 15:22:55.308115 finished: projecting grid cell center arrays from <pyproj

### Export whole packages to a netCDF file

In [6]:
# export mflist
fnc = ml.wel.export(os.path.join(model_ws, 'packages.nc'))
ml.lpf.export(fnc)
fnc.nc

('2016-01-16 15:23:04.386011 WARNING: removing existing nc file: data/packages.nc\n',)
('2016-01-16 15:23:04.386308 starting: initializing attributes\n',)
('2016-01-16 15:23:04.386379 finished: initializing attributes, took: 0:00:00.000071\n',)
('2016-01-16 15:23:04.386501 starting: getting 4D masked arrays for wel\n',)
('2016-01-16 15:23:04.386951 finished: getting 4D masked arrays for wel, took: 0:00:00.000450\n',)
('2016-01-16 15:23:04.396485 starting: creating variable: cumulative_well_flux\n',)
('2016-01-16 15:23:04.396591 starting: initializing geometry\n',)
('2016-01-16 15:23:04.396651 starting: building grid crs using proj4 string: +init=epsg:4326\n',)
('2016-01-16 15:23:04.396714 finished: building grid crs using proj4 string: +init=epsg:4326, took: 0:00:00.000063\n',)
('2016-01-16 15:23:04.396854 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x10896e598> to <pyproj.Proj object at 0x10897f2c0>\n',)
('2016-01-16 15:23:04.396924 finished: projecting gr

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2016-01-16T21:23:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -27.3325004578
    geospatial_vertical_max: -17.3986301422
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 0.0
    origin_y: 10000.0
    origin_crs: epsg:4326
    grid_rotation_from_origin: 0.0
    dimensions(sizes): time(1), layer(1), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), float64 elevation(layer,y,x), float64 longitude(y,x), float64 latitude(y,x), float32 layer(layer), float32 delc(y), float32 delr(x), |S1 VerticalTransform(), float32 cumulative_well_flux(layer,y,x), float32 well_cell_2d(y,x), float32 wel_flux(time,layer,y,x), float32 wel_iface(time,layer,y,x), float32 hani(layer,y,x), float32 hk(layer,y,x), float32 ss(layer,y,x), float32 sy(layer,y,x), float32 vka(layer,y,x), float32 vkcb(layer,y,x), float3

### Export the whole model to a netCDF and a shapefile

In [7]:
fnc = ml.export(os.path.join(model_ws, 'model.nc'))
ml.export(os.path.join(model_ws, 'model.shp'))
fnc.nc

('2016-01-16 15:23:45.049693 WARNING: removing existing nc file: data/model.nc\n',)
('2016-01-16 15:23:45.050047 starting: initializing attributes\n',)
('2016-01-16 15:23:45.050118 finished: initializing attributes, took: 0:00:00.000071\n',)
('2016-01-16 15:23:45.050221 starting: getting 3D array for botm\n',)
('2016-01-16 15:23:45.050311 finished: getting 3D array for botm, took: 0:00:00.000090\n',)
('2016-01-16 15:23:45.050401 starting: creating variable: botm\n',)
('2016-01-16 15:23:45.050449 starting: initializing geometry\n',)
('2016-01-16 15:23:45.050498 starting: building grid crs using proj4 string: +init=epsg:4326\n',)
('2016-01-16 15:23:45.050560 finished: building grid crs using proj4 string: +init=epsg:4326, took: 0:00:00.000062\n',)
('2016-01-16 15:23:45.050717 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x10897f390> to <pyproj.Proj object at 0x10897f808>\n',)
('2016-01-16 15:23:45.050807 finished: projecting grid cell center arrays from <pypro

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2016-01-16T21:23:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -27.3325004578
    geospatial_vertical_max: -17.3986301422
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 0.0
    origin_y: 10000.0
    origin_crs: epsg:4326
    grid_rotation_from_origin: 0.0
    dimensions(sizes): time(1), layer(1), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), float64 elevation(layer,y,x), float64 longitude(y,x), float64 latitude(y,x), float32 layer(layer), float32 delc(y), float32 delr(x), |S1 VerticalTransform(), float32 botm(layer,y,x), float32 thickness(layer,y,x), float32 top(y,x), int32 ibound(layer,y,x), float32 strt(layer,y,x), float32 hani(layer,y,x), float32 hk(layer,y,x), float32 ss(layer,y,x), float32 sy(layer,y,x), float32 vka(layer,y,x), float32 vkcb(layer,y,x), float32 wetdry(layer